In [1]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
)
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest
from google.oauth2 import service_account
import pandas as pd
import psycopg2 as ps

In [3]:
# Путь к JSON-файлу с учетными данными
KEY_FILE_LOCATION = "/Users/elina/Downloads/credentials.json"

# Настройка учетных данных
credentials = service_account.Credentials.from_service_account_file(KEY_FILE_LOCATION)
client = BetaAnalyticsDataClient(credentials=credentials)
PROPERTY_ID = '453715641'

I0000 00:00:1723572575.453638  921126 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [37]:
# Создаем запрос с несколькими метриками
request = RunReportRequest(
    property=f"properties/{PROPERTY_ID}",
    dimensions=[Dimension(name="country"), Dimension(name="date")],  # Выбор измерения: дата
    metrics=[
       Metric(name="activeUsers"),  # Активные пользователи
        Metric(name="newUsers"),  # Новые пользователи
        Metric(name="totalUsers"),  # Общие пользователи
        Metric(name="userEngagementDuration"),  # Длительность взаимодействия
        Metric(name="sessions"),  # Количество сеансов
        Metric(name="bounceRate"),  # Показатель отказов
        Metric(name="averageSessionDuration"),  # Длительность сеанса
        Metric(name="eventCount"),  # Количество событий
        Metric(name="conversions"),  # Количество конверсий
        Metric(name="transactions")  # Транзакции
    ],
    date_ranges=[DateRange(start_date="2024-08-01", end_date="2024-08-31")]
)

In [39]:
# Выполнение запроса
response = client.run_report(request)


In [40]:
# Подготовка данных для DataFrame
data = {
    "country" : [],
    "date": [],
    "active_users": [],
    "new_users": [],
    "total_users": [],
    "engagement_duration": [],
    "sessions": [],
    "bounce_rate": [],
    "session_duration": [],
    "event_count": [],
    "conversions": [],
    "transaction_revenue": []
}

for row in response.rows:
    dimensions = row.dimension_values
    metrics = row.metric_values
    data["country"].append(dimensions[0].value)
    data["date"].append(dimensions[1].value)
    data["active_users"].append(metrics[0].value)
    data["new_users"].append(metrics[1].value)
    data["total_users"].append(metrics[2].value)
    data["engagement_duration"].append(metrics[3].value)
    data["sessions"].append(metrics[4].value)
    data["bounce_rate"].append(metrics[5].value)
    data["session_duration"].append(metrics[6].value)
    data["event_count"].append(metrics[7].value)
    data["conversions"].append(metrics[8].value)
    data["transaction_revenue"].append(metrics[9].value)

# Преобразование данных в DataFrame
df_ga = pd.DataFrame(data)
df_ga.head()

,country,date,active_users,new_users,total_users,engagement_duration,sessions,bounce_rate,session_duration,event_count,conversions,transaction_revenue
0,Germany,20240811,5,5,5,1824,10,0.4,521.96548269999994,401,0,0
1,Russia,20240811,4,3,4,635,5,0,284.2726166,204,0,0
2,(not set),20240811,3,3,3,0,3,1,0,9,0,0
3,Germany,20240812,2,0,2,182,3,0.66666666666666663,258.54335433333335,68,0,0
4,Germany,20240813,2,1,2,224,2,0,2736.2529595,84,6,0


In [49]:
df_ga.dtypes

country                        object
date                   datetime64[ns]
active_users                    int64
new_users                       int64
total_users                     int64
engagement_duration             int64
sessions                        int64
bounce_rate                   float64
session_duration              float64
event_count                     int64
conversions                     int64
transaction_revenue             int64
dtype: object

In [45]:
df_ga['date'] = pd.to_datetime(df_ga['date'])

In [47]:
df_ga = df_ga.astype({'active_users':'int'\
                      , 'new_users':'int'\
                      , 'total_users':'int'\
                      , 'engagement_duration':'int'\
                      , 'sessions': 'int' \
                      , 'bounce_rate' : 'float' \
                      , 'session_duration' : 'float' \
                      , 'event_count': 'int' \
                      , 'conversions' : 'int' \
                      , 'transaction_revenue' : 'int'})

In [ ]:
conn_etl = ps.connect(
    dbname="dbname",
    user="user",
    password="password",
    host="host",
    port="5433"
)

In [ ]:
conn_test = ps.connect(
dbname = "dbname",
user = "user",
password = "password",
host = "host",
port = "5432"
)

In [81]:
# SQL-запрос для создания таблицы
create_table_query = """ 
CREATE TABLE IF NOT EXISTS google_analytics ( 
    date TIMESTAMP,
    country VARCHAR,
    active_users INTEGER,
    new_users INTEGER,
    total_users INTEGER,
    engagement_duration INTEGER,
    sessions INTEGER,
    bounce_rate FLOAT,
    session_duration FLOAT,
    event_count INTEGER,
    conversions INTEGER,
    transaction_revenue INTEGER
) 
"""

# Подключение к базе данных PostgreSQL
conn_etl = ps.connect(
    dbname="etl",
    user="test",
    password="test",
    host="app.tony8.org",
    port="5433"
)

try:
    # Открываем курсор для выполнения SQL-запросов
    cursor = conn_etl.cursor()
    
    # Выполняем SQL-запрос на создание таблицы
    cursor.execute(create_table_query)
    
    # Фиксируем изменения в базе данных
    conn_etl.commit()

    print("Таблица успешно создана")

except Exception as error:
    print("Ошибка при создании таблицы:", error)
    # Откатываем изменения в случае ошибки
    conn.rollback()

finally:
    # Закрываем курсор и соединение с базой данных
    cursor.close()
    conn_etl.close()


Таблица успешно создана


In [83]:
# Добавлю записи в таблицу active_users

# Подключаемся к базе данных
conn_etl = ps.connect(
dbname = "etl",
user = "test",
password = "test",
host = "app.tony8.org",
port = "5433"
)
try:
    # Открываем курсор для выполнения SQL-запросов
    cursor = conn_etl.cursor()

    # Подготавливаем SQL-запрос для вставки данных
    insert_query = """
    INSERT INTO google_analytics (date, country, active_users, new_users, total_users, engagement_duration, sessions, bounce_rate,session_duration,event_count, conversions, transaction_revenue ) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s)
    """
    
    # Преобразуем DataFrame в список кортежей
    data_to_insert = [
        (row['date'], row['country'], row['active_users'], row['new_users'], row['total_users'], row['engagement_duration'], row['sessions'], row['bounce_rate'], row['session_duration'], row['event_count'], row['conversions'], row['transaction_revenue'] ) 
        for index, row in df_ga.iterrows()
    ]
    
    # Выполняем вставку данных
    cursor.executemany(insert_query, data_to_insert)
    
    # Фиксируем изменения в базе данных
    conn_etl.commit()

    print("Данные успешно вставлены в таблицу")

except Exception as error:
    print("Ошибка при вставке данных:", error)
    # Откатываем изменения в случае ошибки
    conn_etl.rollback()

finally:
    # Закрываем курсор и соединение с базой данных
    cursor.close()
    conn_etl.close()


Данные успешно вставлены в таблицу


In [85]:
query_1 = """ SELECT * 
              FROM google_analytics """

In [87]:
conn_etl = ps.connect(
dbname = "etl",
user = "test",
password = "test",
host = "app.tony8.org",
port = "5433"
)

In [89]:
df_from_pg = pd.read_sql_query(query_1, conn_etl)

/var/folders/23/pwxny9r1325fh3nscc5csfkm0000gn/T/ipykernel_42246/1391313561.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_from_pg = pd.read_sql_query(query_1, conn_etl)


In [91]:
df_from_pg

,date,country,active_users,new_users,total_users,engagement_duration,sessions,bounce_rate,session_duration,event_count,conversions,transaction_revenue
0,2024-08-11,Germany,5,5,5,1824,10,0.400000,521.965483,401,0,0
1,2024-08-11,Russia,4,3,4,635,5,0.000000,284.272617,204,0,0
2,2024-08-11,(not set),3,3,3,0,3,1.000000,0.000000,9,0,0
3,2024-08-12,Germany,2,0,2,182,3,0.666667,258.543354,68,0,0
4,2024-08-13,Germany,2,1,2,224,2,0.000000,2736.252960,84,6,0
5,2024-08-12,United States,2,2,2,0,2,1.000000,0.000000,6,0,0
6,2024-08-12,China,1,0,1,3,1,1.000000,0.000000,1,0,0
7,2024-08-13,Russia,1,0,1,71,1,0.000000,1173.379906,29,24,0
